In [ ]:
import json
import os
# from research_lib.utils.data_access_utils import S3AccessUtils
from research.utils.data_access_utils import S3AccessUtils, RDSAccessUtils
from typing import Tuple
import pandas as pd
import cv2
from matplotlib import pyplot as plt
from weight_estimation.dataset import compute_akpd_score
from keras.models import load_model

s3 = S3AccessUtils('/root/data', json.load(open(os.environ['AWS_CREDENTIALS'])))
rds = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))

<h1> Define helper methods </h1>

In [ ]:
def load_params(params):
    print("Loading params...")
    cameraMatrix1 = np.array(params['CameraParameters1']['IntrinsicMatrix']).transpose()
    cameraMatrix2 = np.array(params['CameraParameters2']['IntrinsicMatrix']).transpose()

    distCoeffs1 = params['CameraParameters1']['RadialDistortion'][0:2] + \
                   params['CameraParameters1']['TangentialDistortion'] + \
                   [params['CameraParameters1']['RadialDistortion'][2]]
    distCoeffs1 = np.array(distCoeffs1)

    distCoeffs2 = params['CameraParameters2']['RadialDistortion'][0:2] + \
                   params['CameraParameters2']['TangentialDistortion'] + \
                   [params['CameraParameters2']['RadialDistortion'][2]]
    distCoeffs2 = np.array(distCoeffs2)

    R = np.array(params['RotationOfCamera2']).transpose()
    T = np.array(params['TranslationOfCamera2']).transpose()

    imageSize = (4096, 3000)

    # perform rectification
    (R1, R2, P1, P2, Q, leftROI, rightROI) = cv2.stereoRectify(cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, imageSize, R, T, None, None, None, None, None, cv2.CALIB_ZERO_DISPARITY, 0)

    left_maps = cv2.initUndistortRectifyMap(cameraMatrix1, distCoeffs1, R1, P1, imageSize, cv2.CV_16SC2)
    right_maps = cv2.initUndistortRectifyMap(cameraMatrix2, distCoeffs2, R2, P2, imageSize, cv2.CV_16SC2)

    print("Params loaded.")
    return left_maps, right_maps

IMAGE_WIDTH = 4096
IMAGE_HEIGHT = 3000

def get_camera_parameters(params: dict) -> Tuple:
    """Return individual camera parameters from JSON stereo parameters contents."""
    
    cameraMatrix1 = np.array(params['CameraParameters1']['IntrinsicMatrix']).transpose()
    cameraMatrix2 = np.array(params['CameraParameters2']['IntrinsicMatrix']).transpose()

    distCoeffs1 = params['CameraParameters1']['RadialDistortion'][0:2] + \
                   params['CameraParameters1']['TangentialDistortion'] + \
                   [params['CameraParameters1']['RadialDistortion'][2]]
    distCoeffs1 = np.array(distCoeffs1)

    distCoeffs2 = params['CameraParameters2']['RadialDistortion'][0:2] + \
                   params['CameraParameters2']['TangentialDistortion'] + \
                   [params['CameraParameters2']['RadialDistortion'][2]]
    distCoeffs2 = np.array(distCoeffs2)

    R = np.array(params['RotationOfCamera2']).transpose()
    T = np.array(params['TranslationOfCamera2']).transpose()
    
    imageSize = (IMAGE_WIDTH, IMAGE_HEIGHT)
    (R1, R2, P1, P2, Q, leftROI, rightROI) = cv2.stereoRectify(cameraMatrix1, distCoeffs1, cameraMatrix2, 
                                                               distCoeffs2, imageSize, R, T, None, None, 
                                                               None, None, None, cv2.CALIB_ZERO_DISPARITY, 0)
    left_maps = cv2.initUndistortRectifyMap(cameraMatrix1, distCoeffs1, R1, P1, imageSize, cv2.CV_16SC2)
    right_maps = cv2.initUndistortRectifyMap(cameraMatrix2, distCoeffs2, R2, P2, imageSize, cv2.CV_16SC2)
    
    return left_maps, right_maps, cameraMatrix1, distCoeffs1, R1, P1, cameraMatrix2, distCoeffs2, R2, P2

<h1> Load dataset of unrectified annotations and RGB rectified annotations </h1>

In [ ]:
f = '/root/data/alok/biomass_estimation/playground/2020-03-01_crop_annotation_88_unrectified_akpd.csv'
df1 = pd.read_csv(f, header=None)
df1.columns = ['captured_at', 'pen_id', 'left_crop_url', 'right_crop_url', 'left_crop_metadata', 
             'right_crop_metadata', 'camera_metadata', 'annotation_unrectified']

In [ ]:
df1.left_crop_url.iloc[0]

In [ ]:
df.left_crop_url.iloc[0]

In [ ]:
query = """
select
    * 
from
    prod.biomass_computations
where
    pen_id = 88 and 
    group_id = '88' and
    captured_at between '2020-03-01' and '2020-03-07' and
    camera_metadata is not null
order by 
    captured_at;
"""

df2 = rds.extract_from_database(query)



In [ ]:
df = df2.merge(df1[['left_crop_url', 'annotation_unrectified']], how='inner', on='left_crop_url')

<h1> Load weight estimation modules </h1>

In [ ]:
"""
This module contains constants representing core & auxiliary fish body parts.
"""

UPPER_LIP = 'UPPER_LIP'
EYE = 'EYE'
PECTORAL_FIN = 'PECTORAL_FIN'
DORSAL_FIN = 'DORSAL_FIN'
PELVIC_FIN = 'PELVIC_FIN'
ADIPOSE_FIN = 'ADIPOSE_FIN'
ANAL_FIN = 'ANAL_FIN'
TAIL_NOTCH = 'TAIL_NOTCH'
UPPER_PRECAUDAL_PIT = 'UPPER_PRECAUDAL_PIT'
LOWER_PRECAUDAL_PIT = 'LOWER_PRECAUDAL_PIT'
HYPURAL_PLATE = 'HYPURAL_PLATE'

core_body_parts = sorted([UPPER_LIP,
                          EYE,
                          PECTORAL_FIN,
                          DORSAL_FIN,
                          PELVIC_FIN,
                          ADIPOSE_FIN,
                          ANAL_FIN,
                          TAIL_NOTCH])

auxiliary_body_parts = sorted([UPPER_PRECAUDAL_PIT,
                               LOWER_PRECAUDAL_PIT,
                               HYPURAL_PLATE])

all_body_parts = sorted(core_body_parts + auxiliary_body_parts)

In [ ]:
"""This module contains utility helper functions for the WeightEstimator class."""

from collections import namedtuple
from typing import Dict, List, Tuple
import numpy as np
import torch


CameraMetadata = namedtuple('CameraMetadata',
                            ['focal_length', 'focal_length_pixel', 'baseline_m',
                             'pixel_count_width', 'pixel_count_height', 'image_sensor_width',
                             'image_sensor_height'])


def get_left_right_keypoint_arrs(annotation: Dict[str, List[Dict]]) -> Tuple:
    """Gets numpy array of left and right keypoints given input keypoint annotation.
    Args:
        annotation: dict with keys 'leftCrop' and 'rightCrop'. Values are lists where each element
        is a dict with keys 'keypointType', 'xCrop' (num pixels from crop left edge),
        'yCrop' (num pixels from crop top edge), 'xFrame' (num pixels from full frame left edge),
        and 'yFrame' (num pixels from full frame top edge).
    Returns:
        X_left: numpy array containing left crop (xFrame, yFrame) for each key-point ordered
        alphabetically.
        X_right: same as above, but for right crop.
    """

    left_keypoints, right_keypoints = {}, {}
    for item in annotation['leftCrop']:
        body_part = item['keypointType']
        left_keypoints[body_part] = (item['xFrame'], item['yFrame'])

    for item in annotation['rightCrop']:
        body_part = item['keypointType']
        right_keypoints[body_part] = (item['xFrame'], item['yFrame'])

    left_keypoint_arr, right_keypoint_arr = [], []
    for body_part in core_body_parts:
        left_keypoint_arr.append(left_keypoints[body_part])
        right_keypoint_arr.append(right_keypoints[body_part])

    X_left = np.array(left_keypoint_arr)
    X_right = np.array(right_keypoint_arr)
    return X_left, X_right


def normalize_left_right_keypoint_arrs(X_left: np.ndarray, X_right: np.ndarray) -> Tuple:
    """Normalizes input left and right key-point arrays. The normalization involves (1) 2D
    translation of all keypoints such that they are centered, (2) rotation of the 2D coordiantes
    about the center such that the line passing through UPPER_LIP and fish center is horizontal.
    """

    # translate key-points, perform reflection if necessary
    upper_lip_idx = core_body_parts.index(UPPER_LIP)
    tail_notch_idx = core_body_parts.index(TAIL_NOTCH)
    if X_left[upper_lip_idx, 0] > X_left[tail_notch_idx, 0]:
        X_center = 0.5 * (np.max(X_left, axis=0) + np.min(X_left, axis=0))
        X_left_centered = X_left - X_center
        X_right_centered = X_right - X_center
    else:
        X_center = 0.5 * (np.max(X_right, axis=0) + np.min(X_right, axis=0))
        X_left_centered = X_right - X_center
        X_right_centered = X_left - X_center
        X_left_centered[:, 0] = -X_left_centered[:, 0]
        X_right_centered[:, 0] = -X_right_centered[:, 0]

    # rotate key-points
    upper_lip_x, upper_lip_y = tuple(X_left_centered[upper_lip_idx])
    theta = np.arctan(upper_lip_y / upper_lip_x)
    R = np.array([
        [np.cos(theta), -np.sin(theta)],
        [np.sin(theta), np.cos(theta)]
    ])

    D = X_left_centered - X_right_centered
    X_left_rot = np.dot(X_left_centered, R)
    X_right_rot = X_left_rot - D
    return X_left_rot, X_right_rot


def convert_to_world_point_arr(X_left: np.ndarray, X_right: np.ndarray,
                               camera_metadata: CameraMetadata) -> np.ndarray:
    """Converts input left and right normalized keypoint arrays into world coordinate array."""

    y_world = camera_metadata.focal_length_pixel * camera_metadata.baseline_m / \
              (X_left[:, 0] - X_right[:, 0])

    # Note: the lines commented out below are technically the correct formula for conversion
    # x_world = X_left[:, 0] * y_world / camera_metadata.focal_length_pixel
    # z_world = -X_left[:, 1] * y_world / camera_metadata.focal_length_pixel
    x_world = ((X_left[:, 0] * camera_metadata.image_sensor_width / camera_metadata.pixel_count_width) * y_world) / (camera_metadata.focal_length)
    z_world = (-(X_left[:, 1] * camera_metadata.image_sensor_height / camera_metadata.pixel_count_height) * y_world) / (camera_metadata.focal_length)
    X_world = np.vstack([x_world, y_world, z_world]).T
    return X_world


def stabilize_keypoints(X: np.ndarray) -> np.ndarray:
    """Transforms world coordinate array so that neural network inputs are stabilized"""
    X_new = np.zeros(X.shape)
    X_new[:, 0] = 0.5 * X[:, 0] / X[:, 1]
    X_new[:, 1] = 0.5 * X[:, 2] / X[:, 1]
    X_new[:, 2] = 0.05 / X[:, 1]
    return X_new


def convert_to_nn_input(annotation: Dict[str, List[Dict]], camera_metadata: CameraMetadata) \
        -> torch.Tensor:
    """Convrts input keypoint annotation and camera metadata into neural network tensor input."""
    X_left, X_right = get_left_right_keypoint_arrs(annotation)
    X_left_norm, X_right_norm = normalize_left_right_keypoint_arrs(X_left, X_right)
    X_world = convert_to_world_point_arr(X_left_norm, X_right_norm, camera_metadata)
    X = stabilize_keypoints(X_world)
    nn_input = torch.from_numpy(np.array([X])).float()
    return nn_input

In [ ]:
"""
This module contains the WeightEstimator class for estimating fish weight (g), length (mm), and
k-factor given input keypoint coordinates and camera metadata.
"""

from typing import Dict, Tuple
import torch
from torch import nn


class Network(nn.Module):
    """Network class defines neural-network architecture for both weight and k-factor estimation
    (currently both neural networks share identical architecture)."""

    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(24, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        """Run inference on input keypoint tensor."""
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.output(x)
        return x


class WeightEstimator:
    """WeightEstimator class is used to predict fish weight, k-factor, and length
    given input keypoint annotations and camera metadata."""

    def __init__(self, weight_model_f: str, kf_model_f: str) -> None:
        """Initializes class with input weight and k-factor neural-networks."""
        self.weight_model = Network()
        self.weight_model.load_state_dict(torch.load(weight_model_f))
        self.weight_model.eval()

        self.kf_model = Network()
        self.kf_model.load_state_dict(torch.load(kf_model_f))
        self.kf_model.eval()

    @staticmethod
    def _get_model_input(annotation: Dict, camera_metadata: CameraMetadata) -> torch.Tensor:
        """Generates neural-network input tensor given annotation and camera_metadata."""
        X = convert_to_nn_input(annotation, camera_metadata)
        return X

    def predict_weight(self, annotation: Dict, camera_metadata: CameraMetadata) -> float:
        """Generates weight prediction given input annotation and camera metadata."""
        X = self._get_model_input(annotation, camera_metadata)
        weight = 1e4 * self.weight_model(X).item()
        return weight

    def predict_kf(self, annotation: Dict, camera_metadata: CameraMetadata) -> float:
        """Generates k-factor prediction gievn input annotation and camera metadata."""
        X = self._get_model_input(annotation, camera_metadata)
        kf = self.kf_model(X).item()
        return kf

    def predict(self, annotation: Dict, camera_metadata: CameraMetadata) -> Tuple:
        """Generates weight, k-factor, and length predictions given input annotation and camera
        metadata."""
        weight = self.predict_weight(annotation, camera_metadata)
        kf = self.predict_kf(annotation, camera_metadata)
        if weight * kf > 0:
            length = (1e5 * weight / kf) ** (1.0 / 3)
        else:
            length = 0
        return weight, length, kf

<h1> Perform point-based rectification on unrectified key-points </h1>

In [ ]:
cm = df.camera_metadata.iloc[0]
cm_obj = CameraMetadata(
    focal_length=cm['focalLength'],
    focal_length_pixel=cm['focalLengthPixel'],
    baseline_m=cm['baseline'],
    pixel_count_width=cm['pixelCountWidth'],
    pixel_count_height=cm['pixelCountHeight'],
    image_sensor_width=cm['imageSensorWidth'],
    image_sensor_height=cm['imageSensorHeight']
)

params_f, _, _ = s3.download_from_url(cm['stereoParametersUrl'])
params = json.load(open(params_f))
left_maps, right_maps, cameraMatrix1, distCoeffs1, R1, P1, cameraMatrix2, distCoeffs2, R2, P2 = get_camera_parameters(params)

weight_model_url = 'https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/playground/nn_epoch_798_v2.pb'
kf_model_url = 'https://aquabyte-models.s3-us-west-1.amazonaws.com/k-factor/playground/kf_predictor_v2.pb'
weight_model_f, _, _ = s3.download_from_url(weight_model_url)
kf_model_f, _, _ = s3.download_from_url(weight_model_url)

weight_estimator = WeightEstimator(weight_model_f, kf_model_f)

akpd_scorer_url = 'https://aquabyte-models.s3-us-west-1.amazonaws.com/keypoint-detection-scorer/akpd_scorer_model_TF.h5'
akpd_scorer_f, _, _ = s3.download_from_url(akpd_scorer_url)
akpd_scorer_network = load_model(akpd_scorer_f)


anns, weights, lengths, kfs, akpd_scores = [], [], [], [], []
count = 0
for idx, row in df.iterrows():
    if count % 100 == 0:
        print('Percentage complete: {}%'.format(round(100 * float(count) / df.shape[0], 2)))
    count += 1
    
    ann_u = json.loads(row.annotation_unrectified)
    ann = row.annotation
    left_crop_url = row.left_crop_url
    right_crop_url = row.right_crop_url
    
    left_bbox = [int(x) for x in os.path.basename(left_crop_url).split('.')[0].split('_')[-4:]]
    right_bbox = [int(x) for x in os.path.basename(right_crop_url).split('.')[0].split('_')[-4:]]
    
    
    # un-rectify with matlab params, re-rectify with circular params
    ann_u_r = {'leftCrop': [], 'rightCrop': []}
    for side in ['leftCrop', 'rightCrop']:
        for item in ann_u[side]:
            bp = item['keypointType']
            
            if side == 'leftCrop':
                x = item['xCrop'] + left_bbox[0]
                y = item['yFrame'] + left_bbox[1]
                x_new, y_new = cv2.undistortPoints(np.array([[[x, y]]]).astype(float), 
                                    cameraMatrix1, distCoeffs1, R=R1, P=P1)[0][0]
            elif side == 'rightCrop':
                x = item['xCrop'] + right_bbox[0]
                y = item['yFrame'] + right_bbox[1]
                x_new, y_new = cv2.undistortPoints(np.array([[[x, y]]]).astype(float), 
                                    cameraMatrix2, distCoeffs2, R=R2, P=P2)[0][0]
                
            x_new, y_new = int(round(x_new)), int(round(y_new))
            ann_u_r[side].append({
                'keypointType': bp,
                'xFrame': x_new,
                'yFrame': y_new,
                'xCrop': x_new - (ann[side][0]['xFrame'] - ann[side][0]['xCrop']),
                'yCrop': y_new - (ann[side][0]['yFrame'] - ann[side][0]['yCrop'])
                
            })
    anns.append(ann_u_r)
    weight, length, kf = weight_estimator.predict(ann_u_r, cm_obj)
    akpd_score = compute_akpd_score(akpd_scorer_network, ann_u_r, cm)
    weights.append(weight)
    lengths.append(length)
    kfs.append(kf)
    akpd_scores.append(akpd_score)

    
    


In [ ]:
df['new_ann'] = anns
df['new_weight'] = weights
df['new_length'] = lengths
df['new_kf'] = kfs
df['new_akpd_score'] = akpd_scores

In [ ]:


df[df.akpd_score > 0.9].estimated_weight_g.mean()

In [ ]:
cutoffs = np.arange(0, 11000, 1000)
for low, high in zip(cutoffs, cutoffs[1:]):
    mask = (df.akpd_score > 0.9) & (df.estimated_weight_g >= low) & (df.estimated_weight_g < high)
    mask2 = (df.new_akpd_score > 0.9) & (df.new_weight >= low) & (df.new_weight < high) 
    pct_1 = df[mask].shape[0] / df[df.akpd_score > 0.9].shape[0]
    pct_2 = df[mask2].shape[0] / df[df.new_akpd_score > 0.9].shape[0]
    print(low, high, pct_1, pct_2)

In [ ]:
df[df.new_akpd_score > 0.9].new_weight.mean()

In [ ]:
mask = (df.akpd_score > 0.9) & (df.new_akpd_score > 0.9)
plt.figure(figsize=(15, 8))
plt.scatter(df[mask].estimated_weight_g, df[mask].new_weight)
plt.plot([0, 10000], [0, 10000], color='red')
plt.grid()
plt.show()


In [ ]:
(np.mean((df[mask].estimated_weight_g - df[mask].new_weight)**2))** 0.5 / df[mask].estimated_weight_g.mean()

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize
from scipy.interpolate import interpn

def density_scatter(x, y, bins=20, **kwargs):
    fig, ax = plt.subplots(figsize=(20, 10))
    data, x_e, y_e = np.histogram2d(x, y, bins=bins, density=True)
    z = interpn((0.5*(x_e[1:] + x_e[:-1]), 0.5*(y_e[1:]+y_e[:-1])), data, np.vstack([x, y]).T,
                method="splinef2d", bounds_error=False)

    z[np.where(np.isnan(z))] = 0.0

    # Sort the points by density, so that the densest points are plotted last
    idx = z.argsort()
    x, y, z = x[idx], y[idx], z[idx]

    ax.scatter(x, y, c=z, **kwargs)

    ax.set_xlabel('Pipeline 1 (RGB rectification) prediction (g)')
    ax.set_ylabel('Pipeline 2 (key-point rectificatio) prediction (g)')
    ax.grid()
    ax.set_title('Single fish weight estiamte comparison')
    norm = Normalize(vmin=np.min(z), vmax=np.max(z))
    cbar = fig.colorbar(cm.ScalarMappable(norm=norm), ax=ax)
    cbar.ax.set_ylabel('Density')

    return ax

In [ ]:
density_scatter(df[mask].estimated_weight_g.values, df[mask].new_weight.values)

In [ ]:
1 - df[df.new_akpd_score > 0.9].shape[0] / df.shape[0]

In [ ]:
def display_crops(left_image, right_image, ann):
    fig, axes = plt.subplots(2, 1, figsize=(20, 20))
    axes[0].imshow(left_image)
    axes[1].imshow(right_image)
    
    left_keypoints = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in ann['leftCrop']}
    right_keypoints = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in ann['rightCrop']}
    
    for bp, kp in left_keypoints.items():
        axes[0].scatter([kp[0]], [kp[1]], color='red', s=1)
    for bp, kp in right_keypoints.items():
        axes[1].scatter([kp[0]], [kp[1]], color='red', s=1)


In [ ]:
idx = 1

rectified_left_image_f, _, _ = s3.download_from_url(df.left_crop_url.iloc[idx])
rectified_right_image_f, _, _ = s3.download_from_url(df.right_crop_url.iloc[idx])
rectified_left_image = cv2.imread(rectified_left_image_f)
rectified_right_image = cv2.imread(rectified_right_image_f)
ann = df.annotation.iloc[idx]
display_crops(rectified_left_image, rectified_right_image, ann)

In [ ]:
idx = 1
unrectified_left_image_f, _, _ = s3.download_from_url(df.left_crop_url.iloc[idx].replace('aquabyte-crops', 'aquabyte-frames-resized-inbound'))
unrectified_right_image_f, _, _ = s3.download_from_url(df.right_crop_url.iloc[idx].replace('aquabyte-crops', 'aquabyte-frames-resized-inbound'))
unrectified_left_image = cv2.imread(unrectified_left_image_f)
unrectified_right_image = cv2.imread(unrectified_right_image_f)
unrectified_ann = json.loads(df.annotation_unrectified.iloc[idx])
display_crops(unrectified_left_image, unrectified_right_image, unrectified_ann)

In [ ]:
new_ann = df.new_ann.iloc[idx]
display_crops(rectified_left_image, rectified_right_image, new_ann)

In [ ]:
new_ann['leftCrop'][5]

In [ ]:
ann['leftCrop'][5]